# Forms

> FastHTML form components for API key input and management

In [ ]:
#| default_exp components.forms

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Optional, List, Dict, Any, Callable
from fasthtml.common import Form, Div, Input, Label, Button, Span, P, Select, Option, Details, Summary

# daisyUI imports
from cjm_fasthtml_daisyui.components.data_input.text_input import text_input, text_input_colors
from cjm_fasthtml_daisyui.components.data_input.fieldset import fieldset, label as fieldset_label
from cjm_fasthtml_daisyui.components.data_input.select import select, select_colors
from cjm_fasthtml_daisyui.components.actions.button import btn, btn_colors, btn_sizes
from cjm_fasthtml_daisyui.components.feedback.alert import alert, alert_colors
from cjm_fasthtml_daisyui.components.data_display.card import card, card_body, card_title, card_actions

# Tailwind imports
from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import gap, flex_display, flex_direction, grow
from cjm_fasthtml_tailwind.utilities.spacing import space, m, p
from cjm_fasthtml_tailwind.utilities.sizing import w
from cjm_fasthtml_tailwind.core.base import combine_classes

from cjm_fasthtml_byok.utils.helpers import get_provider_info, format_provider_name
from cjm_fasthtml_byok.core.security import mask_key

In [ ]:
#| export
from fasthtml.common import Div
from fasthtml.jupyter import JupyUvi, HTMX
from cjm_fasthtml_daisyui.core.testing import create_test_app, create_test_page, start_test_server
from cjm_fasthtml_daisyui.core.themes import DaisyUITheme
from IPython.display import display

## Basic Key Input Form

In [ ]:
#| export
def KeyInputForm(
    provider: str = "openai",  # The API provider identifier
    action: Optional[str] = None,  # Form action URL (defaults to /api/keys/{provider})
    method: str = "post",  # HTTP method (default: "post")
    show_help: bool = True,  # Whether to show help text
    custom_placeholder: Optional[str] = None,  # Custom placeholder text
    extra_fields: Optional[List[tuple]] = None  # Additional form fields as [(name, type, placeholder, required), ...]
) -> Form:  # FastHTML Form component
    "Create a form for inputting an API key."
    provider_info = get_provider_info(provider)
    placeholder = custom_placeholder or provider_info['placeholder']
    action_url = action or f"/api/keys/{provider}"
    
    form_fields = []
    
    # Main API key field
    form_fields.append(
        Div(
            Label(
                Span(f"{provider_info['name']} API Key"),
                Input(
                    type="password",
                    name="api_key",
                    placeholder=placeholder,
                    cls=combine_classes(text_input, w.full),
                    required=True,
                    autocomplete="off"
                )
            ),
            P(
                "Your API key will be encrypted and stored securely" if show_help else "",
                cls=str(fieldset_label) if show_help else None
            ) if show_help else None,
            cls=str(fieldset)
        )
    )
    
    # Add extra fields if provided
    if extra_fields:
        for field_name, field_type, field_placeholder, field_required in extra_fields:
            form_fields.append(
                Div(
                    Label(
                        Span(field_name.replace('_', ' ').title()),
                        Input(
                            type=field_type,
                            name=field_name,
                            placeholder=field_placeholder,
                            cls=combine_classes(text_input, w.full),
                            required=field_required
                        )
                    ),
                    cls=str(fieldset)
                )
            )
    
    # Add submit button
    form_fields.append(
        Button(
            "Save Key",
            type="submit",
            cls=combine_classes(btn, btn_colors.primary)
        )
    )
    
    return Form(
        *form_fields,
        method=method,
        action=action_url,
        cls=combine_classes(flex_display, flex_direction.col, gap._4)
    )

In [ ]:
#| eval: false
#| output: false
test_func = KeyInputForm
app, rt = create_test_app(theme=DaisyUITheme.LIGHT)

@rt
def index():
    return create_test_page(test_func.__doc__.title().split('.')[0].strip(), test_func())
server = start_test_server(app)
display(HTMX())
server.stop()

## Multi-Provider Key Form

In [ ]:
#| export
def MultiProviderKeyForm(
    providers: List[str],  # List of provider identifiers
    action: str = "/api/keys",  # Form action URL
    method: str = "post",  # HTTP method
    default_provider: Optional[str] = None  # Initially selected provider
) -> Form:  # FastHTML Form component with provider selection
    "Create a form that allows selecting from multiple providers."
    default = default_provider or (providers[0] if providers else None)
    
    return Form(
        Div(
            # Provider selection
            Div(
                Label(
                    Span("Select Provider"),
                    Select(
                        *[
                            Option(
                                format_provider_name(p),
                                value=p,
                                selected=(p == default)
                            )
                            for p in providers
                        ],
                        name="provider",
                        cls=combine_classes(select, w.full),
                        required=True,
                        id="provider-select",
                        onchange="updatePlaceholder(this)"
                    )
                ),
                cls=str(fieldset)
            ),
            # API key input
            Div(
                Label(
                    Span("API Key"),
                    Input(
                        type="password",
                        name="api_key",
                        placeholder=get_provider_info(default)['placeholder'] if default else "Your API key",
                        cls=combine_classes(text_input, w.full),
                        required=True,
                        id="api-key-input",
                        autocomplete="off"
                    )
                ),
                P("Your API key will be encrypted and stored securely", cls=str(fieldset_label)),
                cls=str(fieldset)
            ),
            cls=combine_classes(flex_display, flex_direction.col, gap._4)
        ),
        Button(
            "Save Key",
            type="submit",
            cls=combine_classes(btn, btn_colors.primary, w.full)
        ),
        method=method,
        action=action,
        cls=combine_classes(flex_display, flex_direction.col, gap._4)
    )

In [ ]:
#| eval: false
#| output: false
test_func = MultiProviderKeyForm
app, rt = create_test_app(theme=DaisyUITheme.LIGHT)

@rt
def index():
    return create_test_page(test_func.__doc__.title().split('.')[0].strip(), test_func(['openai', 'claude', 'gemini']))
server = start_test_server(app)
display(HTMX())
server.stop()

## Key Management Card

In [ ]:
#| export
def KeyManagementCard(
    provider: str,  # Provider identifier
    has_key: bool,  # Whether a key is stored
    masked_key: Optional[str] = None,  # Masked version of the key for display
    created_at: Optional[str] = None,  # When the key was stored
    expires_at: Optional[str] = None,  # When the key expires
    delete_action: Optional[str] = None,  # URL for delete action
    update_action: Optional[str] = None  # URL for update action
) -> Div:  # Card component for key management
    "Create a card component for managing a stored API key."
    provider_info = get_provider_info(provider)
    
    card_content = []
    
    # Title
    card_content.append(
        Div(
            provider_info['name'],
            cls=str(card_title)
        )
    )
    
    if has_key:
        # Show key info
        if masked_key:
            card_content.append(
                P(
                    Span("Current Key: "),
                    Span(masked_key, cls="font-mono")
                )
            )
        
        if created_at:
            card_content.append(
                P(f"Added: {created_at}", cls="text-sm opacity-70")
            )
        
        if expires_at:
            card_content.append(
                P(f"Expires: {expires_at}", cls="text-sm opacity-70")
            )
        
        # Action buttons
        actions = []
        
        if update_action:
            actions.append(
                Form(
                    Input(
                        type="password",
                        name="api_key",
                        placeholder=provider_info['placeholder'],
                        cls=str(text_input),
                        required=True
                    ),
                    Button(
                        "Update",
                        type="submit",
                        cls=combine_classes(btn, btn_colors.primary, btn_sizes.sm)
                    ),
                    method="post",
                    action=update_action,
                    cls=combine_classes(flex_display, gap._2)
                )
            )
        
        if delete_action:
            actions.append(
                Form(
                    Button(
                        "Delete",
                        type="submit",
                        cls=combine_classes(btn, btn_colors.error, btn_sizes.sm),
                        onclick="return confirm('Are you sure you want to delete this key?');"
                    ),
                    method="post",
                    action=delete_action
                )
            )
        
        if actions:
            card_content.append(
                Div(
                    *actions,
                    cls=str(card_actions)
                )
            )
    else:
        # No key stored
        card_content.append(
            P("No key configured", cls="text-sm opacity-70")
        )
        
        # Add key form
        if update_action:
            card_content.append(
                KeyInputForm(
                    provider=provider,
                    action=update_action,
                    show_help=False
                )
            )
    
    return Div(
        Div(
            *card_content,
            cls=str(card_body)
        ),
        cls=str(card)
    )

In [ ]:
#| eval: false
#| output: false
test_func = KeyManagementCard
app, rt = create_test_app(theme=DaisyUITheme.LIGHT)

@rt
def index():
    return create_test_page(test_func.__doc__.title().split('.')[0].strip(), test_func(
    provider="openai",
    has_key=True,
    masked_key="sk-...abc",
    created_at="2 hours ago"
))
server = start_test_server(app)
display(HTMX())
server.stop()

## Key Manager Dashboard

In [ ]:
#| export
def KeyManagerDashboard(
    request,  # FastHTML request object
    providers: List[str],  # List of provider identifiers to manage
    byok_manager = None,
    user_id: Optional[str] = None,  # Optional user ID for database storage
    base_url: str = "/api/keys"  # Base URL for API endpoints
) -> Div:  # Dashboard component with all provider cards
    "Create a complete dashboard for managing multiple API keys."
    # Get BYOK manager from request scope if not provided
    if byok_manager is None and 'byok' in request.scope:
        byok_manager = request.scope['byok']
    
    cards = []
    
    for provider in providers:
        has_key = False
        masked_key = None
        
        if byok_manager:
            # Check if key exists
            has_key = byok_manager.has_key(request, provider, user_id)
            if has_key:
                # Get masked version of key
                actual_key = byok_manager.get_key(request, provider, user_id)
                if actual_key:
                    masked_key = mask_key(actual_key)
        
        cards.append(
            KeyManagementCard(
                provider=provider,
                has_key=has_key,
                masked_key=masked_key,
                delete_action=f"{base_url}/{provider}/delete" if has_key else None,
                update_action=f"{base_url}/{provider}"
            )
        )
    
    return Div(
        Div(
            *cards,
            cls="grid grid-cols-1 md:grid-cols-2 lg:grid-cols-3 gap-4"
        ),
        cls="w-full"
    )

In [ ]:
#| eval: false
#| output: false
test_func = KeyManagerDashboard
app, rt = create_test_app(theme=DaisyUITheme.LIGHT)

@rt
def index():
    # Create a mock request object with simulated keys
    class MockRequest:
        pass
    
    request = MockRequest()
    
    # Simulate having keys for some providers
    providers = ["openai", "anthropic", "google", "groq", "fireworks", "xai"]
    
    # Create mock cards - some with keys, some without
    cards = []
    for i, provider in enumerate(providers):
        has_key = i % 2 == 0  # Every other provider has a key
        cards.append(
            KeyManagementCard(
                provider=provider,
                has_key=has_key,
                masked_key=f"sk-...{provider[:3]}" if has_key else None,
                created_at="2 hours ago" if has_key else None,
                delete_action=f"/api/keys/{provider}/delete" if has_key else None,
                update_action=f"/api/keys/{provider}"
            )
        )
    
    # Create the dashboard layout
    dashboard = Div(
        Div(
            *cards,
            cls="grid grid-cols-1 md:grid-cols-2 lg:grid-cols-3 gap-4"
        ),
        cls="w-full"
    )
    
    return create_test_page("Key Manager Dashboard", dashboard)

server = start_test_server(app)
display(HTMX())
server.stop()

## Inline Key Input

In [ ]:
#| export
def InlineKeyInput(
    provider: str,  # Provider identifier
    input_id: Optional[str] = None,  # HTML ID for the input element
    on_save: Optional[str] = None,  # JavaScript to execute on save (or hx-post URL for HTMX)
    compact: bool = True  # Whether to use compact styling
) -> Div:  # Inline input component
    "Create a compact inline key input component."
    provider_info = get_provider_info(provider)
    input_id = input_id or f"key-input-{provider}"
    
    input_elem = Input(
        type="password",
        placeholder=provider_info['placeholder'],
        cls=combine_classes(
            text_input,
            text_input_colors.primary if compact else None,
            "input-sm" if compact else None
        ),
        id=input_id,
        name="api_key"
    )
    
    save_btn = Button(
        "Save",
        cls=combine_classes(
            btn,
            btn_colors.primary,
            btn_sizes.sm if compact else None
        )
    )
    
    # Add HTMX attributes if on_save looks like a URL
    if on_save and on_save.startswith('/'):
        save_btn.attrs['hx-post'] = on_save
        save_btn.attrs['hx-trigger'] = 'click'
        save_btn.attrs['hx-include'] = f'#{input_id}'
    elif on_save:
        save_btn.attrs['onclick'] = on_save
    
    return Div(
        input_elem,
        save_btn,
        cls=combine_classes(flex_display, gap._2)
    )

In [ ]:
#| eval: false
#| output: false
test_func = InlineKeyInput
app, rt = create_test_app(theme=DaisyUITheme.LIGHT)

@rt
def index():
    return create_test_page(test_func.__doc__.title().split('.')[0].strip(), test_func(
    provider="groq",
    on_save="/api/keys/groq/quick-save"
))
server = start_test_server(app)
display(HTMX())
server.stop()

## Tests

In [ ]:
# Test basic key input form
form = KeyInputForm(provider="openai")
assert form.tag == "form"
assert form.attrs['method'] == "post"
assert form.attrs['action'] == "/api/keys/openai"
print("✓ Basic key input form")

# Test multi-provider form
multi_form = MultiProviderKeyForm(
    providers=["openai", "anthropic", "google"],
    default_provider="anthropic"
)
assert multi_form.tag == "form"
print("✓ Multi-provider form")

# Test key management card
card_with_key = KeyManagementCard(
    provider="openai",
    has_key=True,
    masked_key="sk-...abc",
    created_at="2 hours ago"
)
assert card_with_key.tag == "div"
print("✓ Key management card with key")

card_without_key = KeyManagementCard(
    provider="anthropic",
    has_key=False,
    update_action="/api/keys/anthropic"
)
assert card_without_key.tag == "div"
print("✓ Key management card without key")

# Test inline input
inline = InlineKeyInput(
    provider="groq",
    on_save="/api/keys/groq/quick-save"
)
assert inline.tag == "div"
print("✓ Inline key input")

print("\nAll form component tests passed!")

✓ Basic key input form
✓ Multi-provider form
✓ Key management card with key
✓ Key management card without key
✓ Inline key input

All form component tests passed!


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()